# HSV - Hue, Saturation, Value Colorspace
Stough, DIP

- [skimage's color module](https://scikit-image.org/docs/dev/api/skimage.color.html)
- [What is HSV?](https://en.wikipedia.org/wiki/HSL_and_HSV)
- [Why HSV?](https://dsp.stackexchange.com/questions/2687/why-do-we-use-the-hsv-colour-space-so-often-in-vision-and-image-processing)

The HSV and closely related HSL colorspaces are ways of considering 'color' that 
separate the brightness of a color from its tone and purity. This is useful because we
as humans are highly atuned to *brightness changes* in our perception. So if the RGB space speaks
to the mechanism by which we receive a color stimulus--activation of red, green, and 
blue-centered cones--then the HSV speaks more to how we *perceive* a color.

<img src="https://miro.medium.com/max/1700/1*W30TLUP9avQwyyLfwu7WYA.jpeg" style="width:500px;height:200px"/> <img src="https://upload.wikimedia.org/wikipedia/commons/6/6b/HSL_color_solid_cylinder_saturation_gray.png" style="width:250px;height:200px"/>

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import skimage.color as color

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

import matrix_utils
from vis_utils import (vis_rgb_cube,
                       vis_hists,
                       lab_uniform)

In [ ]:
# IMAGE = 'mountainSpring.jpg'
# IMAGE = 'peppers.png'
IMAGE = 'blueridge.jpg'

# Let's construct our own colormaps
cm_KR = mcolors.LinearSegmentedColormap.from_list('kr', [(0,0,0), (1, 0, 0)], 256)
cm_KG = mcolors.LinearSegmentedColormap.from_list('kg', [(0,0,0), (0, 1, 0)], 256)
cm_KB = mcolors.LinearSegmentedColormap.from_list('kb', [(0,0,0), (0, 0, 1)], 256)

In [ ]:
Irgb = plt.imread('../dip_pics/' + IMAGE)

# Image of uniform random distributed color
# Irgb = np.random.random((100,100,3))

In [ ]:
f, ax = plt.subplots(3,3, figsize = (8,6), sharex=True, sharey=True)

Ihsv = color.rgb2hsv(Irgb)

# Row 0: just the full color image
ax[0][1].imshow(Irgb)
ax[0][0].set_axis_off()
ax[0][2].set_axis_off()

# Row 1: r g b channels
for i in range(3):
    ax[1,i].imshow(Irgb[...,i], cmap=[cm_KR, cm_KG, cm_KB][i])
    ax[1,i].set_title(['Red', 'Green', 'Blue'][i])


# Now to display the HSV space.

cmapNameHue = 'myHue'
colorsH = [(.7,0,0), (.5,.5,0), (0,.7,0), (0,.5,.5), (0,0,.7), (.5, 0, .5), (.7, 0, 0)]
cmHue = mcolors.LinearSegmentedColormap.from_list(cmapNameHue, lab_uniform(colorsH), 256)


ax[2,0].imshow(Ihsv[...,0], cmap=cmHue)
ax[2,0].set_title('H')

ax[2,1].imshow(Ihsv[...,1], cmap='gray')
ax[2,1].set_title('S')

ax[2,2].imshow(Ihsv[...,2], cmap='gray')
ax[2,2].set_title('V')

plt.tight_layout()
f.canvas.set_window_title('RGB vs HSV')
plt.show()

In [ ]:
matrix_utils.arr_info(Ihsv)

&nbsp;

## Let's visualize in 3D

Given the HSV graph above, Hue can be seen as P, and Saturation as R in polar coordinates. Value 
is the Z. Easy. So then, how to [plot in polar coords](https://matplotlib.org/3.1.0/gallery/mplot3d/surface3d_radial.html)?

In [ ]:
def visHSV(I, numpoints = 5000):
    
    Ihsv = color.rgb2hsv(I)
    
    P = 2*np.pi*Ihsv[...,0].ravel()
    R = Ihsv[...,1].ravel()
    Z = Ihsv[...,2].ravel()
    
    # Now transform P,R into X,Y for Euclidean scatter.
    X, Y = R*np.cos(P), R*np.sin(P)
    
    # Still need the RGBs for scatter colors
    # X is the N*M x 3 version of the image.
    Xrgb = np.stack([I[...,i].ravel() for i in range(3)]).T
    
    # Pick a random subset of the pixels to plot. Otherwise, pretty chaotic and slow.
    # https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.choice.html
    randomInds = np.random.choice(np.arange(X.shape[0]), numpoints, replace=False)

    
    fig = plt.figure(figsize=(5,4))
    ax = fig.add_subplot(111, projection='3d')
    
    # point colors
    point_colors = Xrgb[randomInds, :]
    if point_colors.max() > 1:
        point_colors = point_colors-point_colors.min()
        point_colors = point_colors/point_colors.max()

    #Now plot those pixels in the 3d space.
    ax.scatter(X[randomInds], Y[randomInds], Z[randomInds], 
               c=point_colors, depthshade=False)

    #Label the axes.
    ax.set_xlabel('H and S')
    ax.set_ylabel('H and S')
    ax.set_zlabel('Value')

In [ ]:
visHSV(Irgb)

&nbsp;

## Manipulating the HSV components of an image...

In [ ]:
def changeHue(newhue):
    global Ihsv
    change = newhue - Ihsv[...,0].mean()
    
    Ic = Ihsv.copy()
    Ic[...,0] = np.fmod(Ic[...,0]+change, 1) # anything > 1 wraps around 0.
    
    neg = Ic[...,0] < 0
    Ic[neg, 0] = 1 + Ic[neg, 0]
    
    return np.clip(Ic, 0, 1)

In [ ]:
from ipywidgets import VBox, HBox, FloatSlider

plt.ioff()
plt.clf()

hue_mean = Ihsv[...,0].mean()

hue_slider = FloatSlider(
    orientation='horizontal',
    value=hue_mean,
    min=0.00,
    max=1.00,
    step=0.01,
    description='Hue'
)

fig_args = {'num':' ', 'frameon':True, 'sharex':True, 'sharey':True}
fig, ax = plt.subplots(1,2, figsize=(8,3), **fig_args) 

I_h = changeHue(hue_mean)
I_rgb = color.hsv2rgb(I_h)

# display artists I'll update
adisp = ax[0].imshow(Irgb)
rdisp = ax[1].imshow(I_rgb)

ax[0].set_title(f'Original ({hue_mean:.03f})')
rtext = ax[1].set_title(f'Hue: {hue_mean:.03f}')


def update_image(change):
    global Ihsv, I_h, adisp, rdisp, rtext
    
    I_rgb = color.hsv2rgb(changeHue(change.new)) 
    rdisp.set_array(I_rgb)
    rtext.set_text(f'Hue: {change.new:.03f}')
    fig.canvas.draw()
    fig.canvas.flush_events()

hue_slider.observe(update_image, names='value')

plt.tight_layout()

VBox([hue_slider, fig.canvas])
